In [ ]:
import pandas as pd
med = pd.read_excel('../../Data/Medscape/2020-08-27/Memorium_USA_Physicians_2020-08-27.xlsx')
twi = pd.read_excel('../../Data/Medscape/2020-08-27/Twitter_Physicians_2020-08-27.xlsx')
her = pd.read_excel('../../Data/Medscape/2020-08-27/Heroes_Physicians_2020-08-27.xlsx')

In [ ]:
med.columns

In [ ]:
twi.columns

In [ ]:
her.columns

In [ ]:
her_ = her[['NAME', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'NICKNAME',
       'TITLE', 'AGE', 'STATE', 'LINK', 'ME']]
her_['SOURCE']='HelpTheHeroes'
med_ = med[['NAME', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'NICKNAME',
       'TITLE', 'AGE', 'STATE', 'LINK', 'ME']]
med_['SOURCE']='Medscape'
tiw_ = twi[['NAME', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'NICKNAME',
       'TITLE', 'AGE', 'STATE', 'LINK', 'ME']]
tiw_['SOURCE']='Twitter'
all_docs = pd.concat([med_, tiw_,her_]).drop_duplicates('ME', keep='first')

In [1]:
import pyodbc
import pandas as pd
username = 'vigrose'
password = 'Ravenclaw~10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [6]:
query = \
    """
    SELECT
    P.PARTY_ID_FROM as PARTY_ID,
    P.STS_TYPE_ID,
    S.DESC as MEMBERSHIP_STATUS
    FROM
    AMAEDW.PARTY_RLSHIP P, AMAEDW.STS_TYPE S
    WHERE
    P.MBRSHP_YR=2021
    AND
    P.THRU_DT is null
    AND
    P.STS_TYPE_ID=S.STS_TYPE_ID;
    """
MEMBERS = pd.read_sql(con=AMAEDW, sql=query)
MEMBERS.head()

,PARTY_ID,STS_TYPE_ID,MEMBERSHIP_STATUS,MBRSHP_YR
0,1798828,5,Non-Member,2021
1,1798000,5,Non-Member,2021
2,1798001,5,Non-Member,2021
3,1798003,5,Non-Member,2021
4,1798004,5,Non-Member,2021


In [9]:
MEMBERS[MEMBERS.PARTY_ID==16397943]

,PARTY_ID,STS_TYPE_ID,MEMBERSHIP_STATUS,MBRSHP_YR
1457046,16397943,5,Non-Member,2021
2907023,16397943,1,Active,2022
2940741,16397943,1,Active,2023
2952559,16397943,1,Active,2024
2955996,16397943,1,Active,2025


In [ ]:
MEMBERS.groupby(['STS_TYPE_ID','MEMBERSHIP_STATUS']).count()

In [ ]:
import pyodbc
username = 'vigrose'
password = 'Gryffindor~10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)
query = \
    """
    SELECT
    S.DESC as MEMBERSHIP_STATUS,
    M.ME
    FROM
    AMAEDW.PARTY_RLSHIP P, AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.STS_TYPE S
    WHERE
    P.MBRSHP_YR=2021 
    AND
    P.THRU_DT is null
    AND
    P.PARTY_ID_FROM=M.PARTY_ID
    AND
    P.STS_TYPE_ID=S.STS_TYPE_ID
    AND
    S.DESC in ('Active', 'Renewal');
    """
MEMBERS = pd.read_sql(con=AMAEDW, sql=query)
MEMBERS.head()

In [ ]:
MEMBERS.groupby('MEMBERSHIP_STATUS').count()

In [ ]:
MEMBERS[MEMBERS.PARTY_ID!='None']

In [ ]:
len(MEMBERS)

In [ ]:
MEMBERS.DESC='ACTIVE'

In [ ]:
pd.merge(MEMBERS, all_docs, how='right').to_excel("../../Data/Medscape/AMA_Members_COVID.xlsx", index=False)

In [ ]:
all_docs.ME

In [ ]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
all_docs['ME']=fix_me(all_docs.ME)

In [ ]:
query = \
    """
    SELECT *
    FROM
    AMAEDW.PARTY_ID_TO_ME_VW;
    """
party_key = pd.read_sql(con=AMAEDW, sql=query)
party_key.head()

In [ ]:
party_key.to_csv('../../Data/Triangulation/Party_ME.csv', index=False)